25/26 KM6312 Report Analysis Group Five Guys

1. Data collection

In [ ]:
import csv
import json
from DrissionPage import ChromiumPage

f = open(file='data.csv', mode='w', encoding='utf-8', newline='')

csv_writer = csv.DictWriter(f, fieldnames=[
    '职位', '城市', '区域', '商圈', '公司', '薪资', '经验', '学历',
    '领域', '性质', '规模', '技能标签', '福利标签'
])
csv_writer.writeheader()

dp = ChromiumPage()
dp.listen.start('zpgeek/search/joblist.json')
dp.get('https://www.zhipin.com/web/geek/job?query=python&city=101290100')

for page in range(1, 30):
    print(f'正在采集第{page}页的数据内容')
    dp.scroll.to_bottom()

    r = dp.listen.wait()

    json_data = r.response.body


    jobList = json_data['zpData']['jobList']

    for job in jobList:
        dit = {
            '职位': job.get('jobName', ''),
            '城市': job.get('cityName', ''),
            '区域': job.get('areaDistrict', ''),
            '商圈': job.get('businessDistrict', ''),
            '公司': job.get('brandName', ''),
            '薪资': job.get('salaryDesc', ''),
            '经验': job.get('jobExperience', ''),
            '学历': job.get('jobDegree', ''),
            '领域': job.get('brandIndustry', ''),
            '性质': job.get('brandStageName', ''),
            '规模': job.get('brandScaleName', ''),
            '技能标签': ' '.join(job.get('skills', [])),
            '福利标签': ' '.join(job.get('welfareList', [])),
        }
        csv_writer.writerow(dit)
        print(dit)

dp.scroll.to_bottom()


正在采集第1页的数据内容
{'职位': 'python量化开发工程师', '城市': '昆明', '区域': '官渡区', '商圈': '关上', '公司': '云南极购电子商务', '薪资': '15-16K', '经验': '3-5年', '学历': '本科', '领域': '互联网', '性质': '未融资', '规模': '0-20人', '技能标签': 'Python Java C Django Flask Tornado Scrapy Numpy MySQL Redis MongoDB PostgreSQL 架构设计经验 Linux开发/部署经验 量化交易开发经验', '福利标签': '年终奖 五险一金 零食下午茶 保底工资 绩效奖金 全勤奖 工龄奖 生日福利 团建聚餐'}
{'职位': 'Python开发（AI方向）', '城市': '昆明', '区域': '盘龙区', '商圈': '人民东路', '公司': '松赞绿谷', '薪资': '8-13K', '经验': '3-5年', '学历': '本科', '领域': '旅游/景区', '性质': '未融资', '规模': '1000-9999人', '技能标签': 'Python PyTorch TensorFlow Flask Django Linux开发/部署经验 LLM AI Agent Shell脚本编写', '福利标签': '意外险 餐补 宿舍有空调 年终奖 包吃 团建聚餐 有无线网 绩效奖金 包住 节日福利 员工旅游 带薪年假 免费工装 零食下午茶 定期体检 五险一金 生日福利'}
{'职位': '高级python开发工程师', '城市': '昆明', '区域': '西山区', '商圈': '前卫', '公司': '岩创科技', '薪资': '7-12K', '经验': '3-5年', '学历': '本科', '领域': '大数据', '性质': '未融资', '规模': '0-20人', '技能标签': 'PyTorch Docker MySQL 机器学习经验 接受无前端经验/技能 MongoDB 其他 架构设计经验 Python Flask TensorFlow', '福利标签': '全勤奖 包住 生日福利 年终奖 带薪年假 交通补助 节日福利 五险'}
{'职位': 'python程

2. Data Cleaning

In [4]:
import pandas as pd

df = pd.read_csv("/Users/wangyouqi/Desktop/ntu/csv/merged.csv")
df = df[df['薪资'] != '面议']
df = df[~df['职位'].str.contains('实习', case=False, na=False)]
df = df.reset_index(drop=True)
df.to_csv("/Users/wangyouqi/Desktop/ntu/csv/cleaned_merged.csv", index=False)

print("cleaned_merged.csv")

cleaned_merged.csv


In [6]:
import pandas as pd
import re

df = pd.read_csv("/Users/wangyouqi/Desktop/ntu/csv/cleaned_merged.csv")

def parse_salary(s):
    if pd.isna(s):
        return pd.Series([None, None, None])

    text = str(s).lower().replace("／", "/").replace("每月", "/月").replace("月薪", "/月")

    if "面议" in text or text.strip() == "":
        return pd.Series([None, None, None])

    match = re.findall(r"(\d+\.?\d*)\s*[-~—－–至]?\s*(\d+\.?\d*)?\s*k", text)
    if match:
        low, high = match[0]
        low = float(low)
        high = float(high) if high else low
        avg = (low + high) / 2 * 1000 
    else:
        avg = None

    if "年" in text:
        avg = (avg or 0) / 12 

    bonus_match = re.search(r"(\d+)\s*薪", text)
    bonus = int(bonus_match.group(1)) if bonus_match else 12

    annual = (avg or 0) * bonus

    return pd.Series([avg, annual, bonus])

df[["monthly_salary_avg", "annual_salary_equiv", "bonus_months"]] = df["薪资"].apply(parse_salary)

df.to_csv("/Users/wangyouqi/Desktop/ntu/csv/adjusted_salary.csv", index=False)

print("adjusted_salary.csv")

adjusted_salary.csv


In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/wangyouqi/Desktop/ntu/csv/adjusted_salary.csv")

df = df.dropna(subset=["monthly_salary_avg", "annual_salary_equiv"])
df = df[(df["monthly_salary_avg"] != 0) & (df["annual_salary_equiv"] != 0)]

df = df.reset_index(drop=True)

df.to_csv("/Users/wangyouqi/Desktop/ntu/csv/final_salary_cleaned.csv", index=False, encoding="utf-8-sig")

print("final_salary_cleaned.csv")

final_salary_cleaned.csv


In [10]:
import pandas as pd

df = pd.read_csv("/Users/wangyouqi/Desktop/ntu/csv/final_salary_cleaned.csv")

df = df.sort_values(by="monthly_salary_avg", ascending=False)

df = df.reset_index(drop=True)

df.to_csv("/Users/wangyouqi/Desktop/ntu/csv/final_salary_sorted.csv", index=False, encoding="utf-8-sig")

print("final_salary_sorted.csv")

final_salary_sorted.csv
